In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

tokenizer = AutoTokenizer.from_pretrained("kreimben/CodeMind-gemma")
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained("kreimben/CodeMind-gemma")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model = model.to('cuda')

In [3]:
get_completion(
    'i dont know about word break problem. please give me a approch about this problem.',
    model=model,
    tokenizer=tokenizer,
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


"user Below is an instruction that describes a task. Write a response that appropriately completes the request. i dont know about word break problem. please give me a approch about this problem.\nmodel model# Intuition\\n<!-- Describe your first thoughts on how to solve this problem. -->\\n\\n# Approach\\n<!-- Describe your approach to solving the problem. -->\\n\\n# Complexity\\n- Time complexity:\\n<!-- Add your time complexity here, e.g. $$O(n)$$ -->\\n\\n- Space complexity:\\n<!-- Add your space complexity here, e.g. $$O(n)$$ -->\\n\\n# Code\\n```\\nclass Solution {\\npublic:\\n    int canBreak(string s) {\\n        vector<int> dp(s.size(), 0);\\n        vector<int> can(s.size(), 0);\\n        dp[0] = 1;\\n        can[0] = 1;\\n        for(int i = 1; i < s.size(); i++) {\\n            for(int j = 0; j < i; j++) {\\n                if(s[j] == s[i] && dp[j] == 1) {\\n                    can[i] = 1;\\n        }else if(i - j > 1 && can[j] == 1) {\\n                    can[i] = 1;\\n   